In [10]:
import airwriting as air
import network_models as nm
import numpy as np
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import explib as exp

In [12]:
#데이터 읽어오기
folderpath_root = '/Data'

bUseNpyData = 1
if bUseNpyData==0:
    [data, len_list] = air.load_chars(folderpath_root) #데이터 읽어오기 [nSubjects, nTrials, nChars, nMaxLen, nChannel]
    np.save('data18.npy',data)
    np.save('len_list18.npy',len_list)
else:
    data = np.load('data18.npy')
    len_list = np.load('len_list18.npy')

In [21]:
import tensorflow as tf

print(tf.__version__)

2.10.0


In [13]:
#데이터의 길이가 다른 trial에 비해 매우 낮은 것 삭제
cnt_deleted = 0
for i in range(data.shape[0]):
    for k in range(data.shape[2]):
        len_mid = np.median(len_list[i,:,k])
        for j in range(data.shape[1]):
            if len_list[i,j,k]< len_mid/2:
                len_list[i,j,k] = 0
                data[i,j,k,:,:] = np.nan
                print(f'deleting... {i}/{j}/{k}')
                cnt_deleted+=1
print(f'{cnt_deleted} letters are deleted')

data = air.process_trim(data)
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        for k in range(data.shape[2]):
            len_list[i,j,k] = data.shape[3] - np.sum(np.isnan(data[i,j,k,:,0]))

# 특징 계산 및 데이터 확장
po = air.get_power_signals(data, len_list)
data = np.concatenate((data,np.expand_dims(po,axis=-1)),axis=4)

deleting... 1/4/3
deleting... 2/4/9
deleting... 4/4/3
deleting... 4/2/14
deleting... 5/4/14
deleting... 5/2/16
deleting... 9/3/9
deleting... 11/4/20
deleting... 11/4/27
deleting... 11/3/34
deleting... 12/3/11
deleting... 13/4/4
deleting... 14/4/25
deleting... 15/4/3
deleting... 15/4/17
deleting... 15/3/25
deleting... 15/4/33
deleting... 16/3/13
18 letters are deleted


In [14]:
#길이를 모두 동일하게 리사이즈
len_to = np.int16(np.max(len_list))  #데이터의 길이 결정
data = air.resize_data(data, len_to)

#정규화
data= air.normalize(data)

#타겟 데이터 만들기
Y = exp.getTargetData(data, bRemoveSameShapeInfo = True)

In [18]:
###############################################
#  N-Fold Validation 시작
nSubjects = data.shape[0]
nTrials = data.shape[1]
nChars = data.shape[2]
nArrayLen = data.shape[3]
nChannels = data.shape[4]

nModels = 5
nModels_sel = 5

cm = np.zeros([nSubjects,nTrials, nChars, nChars])
cm_merged = np.zeros([nChars, nChars])

import random

for i in range(nSubjects):
    print(f'{i} fold:------------------')

    #데이터 분할
    [XTrain, XTest, YTrain, YTest] = exp.spilt_data_for_exp_Nfold(data, Y, i )
    #[XTrain, XTest, YTrain, YTest] = exp.spilt_data_for_exp(data, Y)

    idx = np.array(range(XTrain.shape[0]))
    random.shuffle(idx)
    XTrain = XTrain[idx]
    YTrain = YTrain[idx]

    #없는 데이터 삭제 & 채널 선택
    idx = np.logical_not(np.sum(np.sum(np.isnan(XTrain),-1),-1)>0)
    #XTrain = XTrain[idx,:,:][:,:,[0,1,2,9]]
    XTrain = XTrain[idx, :, :][:, :, :]
    YTrain = YTrain[idx,:]
    idx = np.logical_not(np.sum(np.sum(np.isnan(XTest),-1),-1)>0)
    #XTest = XTest[idx,:,:][:,:,[0,1,2,9]]
    XTest = XTest[idx, :, :][:, :, :]
    YTest = YTest[idx,:]

    #반복해서 학습
    model = []
    hist = []
    val_accuracy = np.zeros(nModels)
    cnt_model = 0
    for j in range(nModels):
        # 모델 선택
        model_tmp = nm.get_model_tree_v2(XTrain.shape[1], XTrain.shape[2], YTrain.shape[-1])
        model_tmp.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001, amsgrad=True),
                      metrics=['accuracy'])
        hist_tmp = model_tmp.fit(x=XTrain, y=YTrain, epochs=400, validation_split=0.2, batch_size=16,verbose=False)

        model.append(model_tmp)
        hist.append(hist_tmp)
        val_accuracy[j] = hist[cnt_model].history['val_accuracy'][-1]
        print(f'    {i}/{j}:{val_accuracy[j]}')

#        model.append(nm.get_model_tree_v2(XTrain.shape[1],XTrain.shape[2], YTrain.shape[-1]))
#        model[cnt_model].compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001, amsgrad=True), metrics=['accuracy'])
#        hist.append(model[cnt_model].fit(x=XTrain, y=YTrain, epochs=100, validation_split = 0.2, batch_size=16, verbose=True))

 #       cnt_model += 1

    # nModels_sel개의 모델 선택
    rank_acc = val_accuracy.argsort()
    cnt = 0
    model_sel = []
    for j in range(nModels):
        if rank_acc[j]<nModels_sel:
            model_sel.append(model[j])
            cnt += 1

#    model_sel = model[rank_acc[:nModels_sel]]

    #predict
    o = np.zeros([XTest.shape[0],YTest.shape[1], nModels_sel])
    print(f'     accuracy each')
    for j in range(nModels_sel):
        o[:,:,j] = model_sel[j].predict(XTest)
        acc_tmp = np.sum(np.argmax(o[:, :, j], -1) == np.argmax(YTest,-1))/ YTest.shape[0]
        print(f'{acc_tmp:.4f}  ', end='') #각각의 accuracy 체크
    print('')

    #정확도 계산
    o = np.median(o, axis=-1)
    o = np.argmax(o, -1)
    YTest_int = np.argmax(YTest,-1)
    acc = np.sum(o == YTest_int)/ o.shape[0]
    print(f'Test Acc (median): {acc}')

    #confusion matrix 계산
    cnt = 0
    for j in range(0, nTrials):
        for k in range(0, nChars):
            if cnt>=YTest.shape[0]:
                break
            cm[i, j, int(YTest_int[cnt]), int(o[cnt])] += 1
            cnt += 1

#   n-Fold Validation 끝
##################################################

0 fold:------------------
    0/0:0.8965517282485962
    0/1:0.8965517282485962
    0/2:0.8965517282485962
    0/3:0.8965517282485962
    0/4:0.8965517282485962
     accuracy each
6/6 [==============================] - 0s 4ms/step
0.9222  
Test Acc (median): 0.9277777777777778
1 fold:------------------
    1/0:0.9014778137207031
    1/1:0.9014778137207031
    1/2:0.9014778137207031
    1/3:0.9014778137207031
    1/4:0.9014778137207031
     accuracy each
6/6 [==============================] - 0s 3ms/step
0.8101  
Test Acc (median): 0.8156424581005587
2 fold:------------------
    2/0:0.8932676315307617
    2/1:0.8932676315307617
    2/2:0.8932676315307617
    2/3:0.8932676315307617
    2/4:0.8932676315307617
     accuracy each
6/6 [==============================] - 0s 5ms/step
0.8994  
Test Acc (median): 0.9106145251396648
3 fold:------------------
    3/0:0.8981937766075134
    3/1:0.8981937766075134
    3/2:0.8981937766075134
    3/3:0.8981937766075134
    3/4:0.8981937766075134
     